# Prototype with variables inputs (FBProphet Model)

In [1]:
#Defining the Function
def function_FBProphet_Forecast(column_name, no_of_days, start_date, end_date):
    import warnings
    warnings.filterwarnings('ignore')
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    
    data = pd.read_csv('ISA_Processed.txt', delimiter = "\t", header=[0,1])
    #formatting column names
    data.columns = data.columns.map('_'.join)
    #print(data)
    #removing unnecessary parts from the column names and rename it
    data=data.rename(columns={'Unnamed: 0_level_0_Date':'Date','Unnamed: 1_level_0_        Time':'Time','Unnamed: 17_level_0_Rain':'Rain',
                         'Unnamed: 16_level_0_Bar  ':'Bar','Unnamed: 33_level_0_ET ':'ET','        Temp_Out':'Temp_Out'})
    
    #dropping some columns temporarily for type fixing of the other columns
    data1=data.drop(['Date','Time','Wind_Dir','Hi_Dir'],axis=1)
    #Replacing '---' with a small value
    data1=data1.replace('---', 0.0001)
    #casting the data to float
    data1=data1.astype(float)

    #formatting time column in original df
    data1['Time'] = data['Time'].str.replace('a|p', '')
    data1['Date']=data['Date']

    data1['DateTime']=data1['Date']+" "+data1['Time']
    data1=data1.set_index('DateTime')
    #data1


    #importing prophet model
    from prophet import Prophet
    import datetime

    
    #Extracting two columns needed for prophet model: 'Date' and target column 'ET'
    data1.index = pd.to_datetime(data1.index, errors='coerce')
    data1.index = data1.index.strftime('%Y/%m/%d %H:%M')
    df=data1
    df['y']=data1[column_name]
    df['ds']=data1.index
    df=df.dropna()

    #creating and fitting prophet model on data
    model = Prophet(interval_width=0.95,yearly_seasonality=True)
    model.fit(df)
    
    no_of_days=int(no_of_days)
    #create future dates
    future_dates = model.make_future_dataframe(periods=no_of_days*24,freq = 'H',include_history=False) #hourly frequency like our data, And future 720 hours(30 days) to be predicted.
    #future_dates.tail() #last 5 values of future data

    
    #forecasting for future dates
    forecast = model.predict(future_dates)
    #filter future dates
    print('Filtered Forecasted Results for '+""+start_date+" to "+end_date)
    forecast_filtered=forecast.loc[(forecast['ds']>=start_date) & (forecast['ds']<=end_date)]
    if forecast_filtered.empty==False:
        print('Forecast For '+""+column_name)
        print(forecast_filtered[['ds', 'yhat']])
        print()
        #Daily Mean
        print('The Daily Mean of the Filtered Forecast Dates:')
        forecast_filtered['ds']= pd.to_datetime(forecast_filtered['ds'])
        forecast_filtered['ds'] = forecast_filtered['ds'].dt.date
        print()
        print('Mean Daily Results for Column: '+""+column_name)
        print(forecast_filtered.groupby('ds')['yhat'].mean())
    #exception
    else:
        print('Enter a range within the forecasted dates and try again!')  

In [2]:
#inputs
column_name=input("Enter a Column Name to Predict(Temp_out/Out_Hum/Wind_Speed/Solar_Rad/ET/ETP:")
no_of_days=input('No of Future Days you want to predict: ')
start_date=input('Enter Start Date of Forecast View(Ex:2020-05-17): ') 
end_date=input('Enter End Date of Forecast View(Ex:2020-08-18): ')


Enter a Column Name to Predict(Temp_out/Out_Hum/Wind_Speed/Solar_Rad/ET/ETP:Temp_Out
No of Future Days you want to predict: 365
Enter Start Date of Forecast View(Ex:2020-05-17): 2020-10-28
Enter End Date of Forecast View(Ex:2020-08-18): 2020-11-17


In [3]:
#Calling function to view outputs
function_FBProphet_Forecast(column_name, no_of_days, start_date, end_date)

Forecasted Results for 2020-10-28 to 2020-11-17
Forecast For Temp_Out
                      ds       yhat
5388 2020-10-28 00:00:00  27.784855
5389 2020-10-28 01:00:00  28.806853
5390 2020-10-28 02:00:00  28.951208
5391 2020-10-28 03:00:00  28.779611
5392 2020-10-28 04:00:00  28.457372
5393 2020-10-28 05:00:00  27.841040
5394 2020-10-28 06:00:00  26.960052
5395 2020-10-28 07:00:00  26.236612
5396 2020-10-28 08:00:00  26.155124
5397 2020-10-28 09:00:00  26.764313
5398 2020-10-28 10:00:00  27.620596
5399 2020-10-28 11:00:00  28.301510
5400 2020-10-28 12:00:00  28.944961
5401 2020-10-28 13:00:00  30.152830
5402 2020-10-28 14:00:00  32.224620
5403 2020-10-28 15:00:00  34.451071
5404 2020-10-28 16:00:00  35.287643
5405 2020-10-28 17:00:00  33.464576
5406 2020-10-28 18:00:00  29.158825
5407 2020-10-28 19:00:00  24.165643
5408 2020-10-28 20:00:00  20.806800
5409 2020-10-28 21:00:00  20.401587
5410 2020-10-28 22:00:00  22.510071
5411 2020-10-28 23:00:00  25.484525
5412 2020-10-29 00:00:00  27.7

In [ ]:
#End